# Week 4 — Wrangling with pandas (Starter Notebook)

**Datasets:** `nyc_taxi_sample.csv`, `taxi_zones.csv`, `covid_daily.csv`

This notebook mirrors the lecture flow: import → inspect → clean → feature engineering → group/reshape → quick visuals.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)


## 1. Load & Inspect

In [ ]:
taxi = pd.read_csv("nyc_taxi_sample.csv",
                    parse_dates=["tpep_pickup_datetime","tpep_dropoff_datetime"])
zones = pd.read_csv("taxi_zones.csv")
covid = pd.read_csv("covid_daily.csv", parse_dates=["date"])

display(taxi.head())
display(taxi.info())
display(zones.head())
display(covid.head())

## 2. Filter, Missing Values, and Basic Cleaning

In [ ]:
# Keep trips in reasonable bounds
taxi_clean = taxi.copy()
taxi_clean = taxi_clean.dropna(subset=["trip_distance"])
taxi_clean = taxi_clean[
    (taxi_clean["trip_distance"].between(0.1, 60)) &
    (taxi_clean["fare_amount"].between(0, 250))
].copy()

# Feature engineering: hour and weekday
taxi_clean["pickup_hour"] = taxi_clean["tpep_pickup_datetime"].dt.hour
taxi_clean["weekday"] = taxi_clean["tpep_pickup_datetime"].dt.day_name()

# Ordered categoricals for weekday
cats = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
taxi_clean["weekday"] = pd.Categorical(taxi_clean["weekday"], categories=cats, ordered=True)

taxi_clean.head()

## 3. Join with Lookup (borough)

In [ ]:
with_boro = taxi_clean.merge(
    zones[["LocationID","borough"]],
    left_on="PULocationID", right_on="LocationID", how="left", validate="m:1"
)
with_boro.head()

## 4. GroupBy & Aggregate

In [ ]:
by_hour = with_boro.groupby("pickup_hour").agg(
    trips=("trip_distance","count"),
    mean_dist=("trip_distance","mean"),
    mean_fare=("fare_amount","mean")
).reset_index()

by_weekday = with_boro.groupby("weekday").size().rename("trips")

display(by_hour.head())
display(by_weekday)

## 5. Quick Visuals with matplotlib

In [ ]:
# Bar: trips by hour
ax = by_hour.plot(x="pickup_hour", y="trips", kind="bar")
ax.set(title="Trips by Pickup Hour", xlabel="Hour", ylabel="Trips")
plt.tight_layout()
plt.show()

In [ ]:
# Bar: trips by weekday
ax = by_weekday.plot(kind="bar")
ax.set(title="Trips by Weekday", xlabel="", ylabel="Trips")
plt.tight_layout()
plt.show()

## 6. Reshape to Pivot → Heatmap

In [ ]:
pivot_md = with_boro.pivot_table(
    index="weekday", columns="pickup_hour",
    values="trip_distance", aggfunc="mean"
)

plt.imshow(pivot_md.values, aspect="auto")
plt.xticks(range(pivot_md.shape[1]), pivot_md.columns)
plt.yticks(range(pivot_md.shape[0]), pivot_md.index)
plt.title("Mean Trip Distance (mi) by Weekday × Hour")
plt.colorbar(label="Miles")
plt.tight_layout()
plt.show()

## 7. COVID-19 (NY) Resampling & Rolling

In [ ]:
state = covid[covid["state"]=="NY"].set_index("date").sort_index()
state["new_cases"] = state["cases"].diff()

weekly = state["new_cases"].resample("W-SUN").sum()
rolling = state["new_cases"].rolling(7, min_periods=1).mean()

ax = weekly.plot()
ax.set(title="NY New Cases (Weekly Sum)", ylabel="Cases")
plt.tight_layout()
plt.show()

ax = rolling.plot()
ax.set(title="NY New Cases (7-Day Rolling Mean)", ylabel="Cases")
plt.tight_layout()
plt.show()

## 8. Mini‑Challenge
**Compute mean fare by borough × hour** and visualize as a heatmap.

Steps:
1) Group & aggregate to mean fare.
2) Pivot to borough rows × hour columns.
3) Plot with `plt.imshow`.


In [ ]:
# Your work here
mean_fare = "try it here"
pivot_fare = "try it here"

plt.imshow(pivot_fare.values, aspect="auto")
plt.xticks(range(pivot_fare.shape[1]), pivot_fare.columns)
plt.yticks(range(pivot_fare.shape[0]), pivot_fare.index)
plt.title("Mean Fare ($) by Borough × Hour")
plt.colorbar(label="$")
plt.tight_layout()
plt.show()

## 9. Homework Scaffold
- Choose a messy dataset (or continue with taxi / covid).
- Demonstrate **groupby aggregate** and **reshaping**.
- Include either **time resampling/rolling** or **ordered categoricals**.
- Produce three well‑labeled charts.


In [11]:
# Template cell for your homework
# df = pd.read_csv("/mnt/data/your_dataset.csv", parse_dates=[...])
# 1) Clean
# 2) Feature engineer
# 3) Group/reshape
# 4) Visualize (matplotlib)
